In [2]:
from idlelib.sidebar import get_end_linenumber

import pandas as pd

In [3]:
# NHANES DATABASES links
# https://wwwn.cdc.gov/nchs/nhanes/continuousnhanes/default.aspx?Cycle=2017-2020
# https://wwwn.cdc.gov/nchs/nhanes/continuousnhanes/default.aspx?BeginYear=2013

# LABEL
# OSQ060 - ever diagnosed with osteoporosis?

# PARAMETERS

# Osteoporosis P_OSQ
# SEQN - patient number
# OSQ160a - did mother have osteoporosis? 1-mother
# OSQ160b - did father have osteoporosis? 2-father

# Demographic P_DEMO
# RIDAGEYR - Age
# RIAGENDR - Gender 1-male, 2-female
# RIDRETH3 - Ethnicity

# Examination P_BMX
# BMXWT - weight
# BMXHT - height
# BMXBMI - bmi

# Dietary P_DR1TOT
# DR1TVD - vitamin D intake
# DR1TCALC - calcium intake

# Laboratory P_BIOPRO
# LBXSCA - total calcium mg/dL

# Alcohol usage P_ALQ
# ALQ121 - alcohol consumption in past 12 months
# ALQ130 - avg alcoholic drinks/day - past 12 months

# Smoking P_SMQ
# SMQ621 - amount of smoked cigarettes
# SMD030 - age at which started smoking regularly

# Reproductive health P_RHQ
# RHD043 - reason for not having regular periods (7 - menopause)
# RHQ060 - age at last menstrual period
# RHQ540 - ever used female hormones? (excludes birth control and use for infertility) 1-yes, 2-no

# Excercise P_PAQ
# PAD615 - vigorous work in minutes on a work day
# PAD660 - vigorous excercise in minutes in a day
# PAD675 - moderate excercise in minutes in a day
# PAD680 - sedentary activity in minutes in a day
# PAD645 - walk/bike in minutes in a day


# not sure if we will use:
    
# OSQ020a - times broken/fractured a hip
# OSQ020b - times broken/fractured a wrist
# OSQ020c - times broken/fractured spine
# OSD050aa - reason for hip fracture
# OSD050ba - reason wrist fracture
# OSD050ca - reason spine fracture

In [44]:
folders = ["2017-2020"]
files = ["OSQ", "DEMO", "BMX", "ALQ", "BIOPRO", "DR1TOT", "PAQ", "RHQ", "SMQ"]
codes = ['SEQN', 'OSQ060', 'OSQ160A', 'OSQ160B', 'ALQ121', 'ALQ130', 'SMQ621', 'SMD030', 'RHD043', 'RHQ060', 'RHQ540', 'PAD615', 'PAD660', 'PAD675', 'PAD680', 'PAD645', 'RIDAGEYR', 'RIAGENDR', 'RIDRETH3', 'BMXWT', 'BMXHT', 'BMXBMI', 'DR1TVD', 'DR1TCALC', 'LBXSCA']
labels_mapping = {
    'SEQN': 'patient',
    'OSQ060': 'osteoporosis',
    'OSQ160A': 'mother osteoporosis',
    'OSQ160B': 'father osteoporosis',
    'ALQ121': 'alcohol past 12 mos',
    'ALQ130': 'avg alcohol/day past 12 mos',
    'SMQ621': 'cigarettes smoked',
    'SMD030': 'regular smoking age',
    'RHD043': 'menopause',
    'RHQ060': 'age of last period',
    'RHQ540': 'female HRT',
    'PAD615': 'physical work (min)',
    'PAD660': 'vigorous excercise (min)', 
    'PAD675': 'moderate excercise (min)',
    'PAD680': 'sedentary (min)',
    'PAD645': 'walk/bike (min)',
    'RIDAGEYR': 'age',
    'RIAGENDR': 'gender',
    'RIDRETH3': 'ethnicity',
    'BMXWT': 'weight',
    'BMXHT': 'height',
    'BMXBMI': 'BMI',
    'DR1TVD': 'vit.D intake',
    'DR1TCALC': 'calcium intake',
    'LBXSCA': 'total calcium mg/dL'
}

data = pd.DataFrame(columns=['patient', 'osteoporosis', 'age', 'gender', 'ethnicity', 'weight', 'height', 'BMI', 'menopause', 'age of last period', 'female HRT',  'vit.D intake', 'calcium intake', 'total calcium mg/dL', 'alcohol past 12 mos', 'avg alcohol/day past 12 mos', 'cigarettes smoked', 'regular smoking age', 'physical work (min)', 'vigorous excercise (min)', 'moderate excercise (min)', 'sedentary (min)', 'walk/bike (min)', 'mother osteoporosis', 'father osteoporosis', ])
for folder in folders:
    for file in files:
        file_path = f"{folder}/{file}.xpt"
        # file_path = "2015-2016/SMQ.xpt"
        df = pd.read_sas(file_path, format="xport")
        filtered = df.filter(items = codes)
        filtered.rename(columns=labels_mapping, inplace=True)
        data = pd.concat([data, filtered], ignore_index=True)
        data = data.groupby('patient', as_index=False).first()
    

# data.dropna(axis=0, how='any', inplace=True)
data = data.dropna(subset=["osteoporosis", "vit.D intake", "calcium intake", "total calcium mg/dL", "age", "gender", "ethnicity", "weight", "height", "alcohol past 12 mos"])
data = data.dropna(thresh=10)



data.dropna(subset=['osteoporosis'], inplace=True)
print(data.shape)
data
# 
# patient_rows = data[data['patient'] == 109290.0]
# patient_rows


C:\Users\omord\AppData\Local\Temp\ipykernel_7112\3014401440.py:40: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  data = pd.concat([data, filtered], ignore_index=True)
C:\Users\omord\AppData\Local\Temp\ipykernel_7112\3014401440.py:40: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  data = pd.concat([data, filtered], ignore_index=True)
C:\Users\omord\AppData\Local\Temp\ipykernel_7112\3014401440.py:40: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. 

(3361, 25)


C:\Users\omord\AppData\Local\Temp\ipykernel_7112\3014401440.py:40: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  data = pd.concat([data, filtered], ignore_index=True)
C:\Users\omord\AppData\Local\Temp\ipykernel_7112\3014401440.py:40: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  data = pd.concat([data, filtered], ignore_index=True)
C:\Users\omord\AppData\Local\Temp\ipykernel_7112\3014401440.py:40: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. 

,patient,osteoporosis,age,gender,ethnicity,weight,height,BMI,menopause,age of last period,...,avg alcohol/day past 12 mos,cigarettes smoked,regular smoking age,physical work (min),vigorous excercise (min),moderate excercise (min),sedentary (min),walk/bike (min),mother osteoporosis,father osteoporosis
11,109274.0,2.0,68.0,1.0,7.0,103.7,185.3,30.2,NaN,NaN,...,2.0,NaN,NaN,480.0,NaN,60.0,300.0,60.0,NaN,NaN
19,109282.0,2.0,76.0,1.0,3.0,83.3,177.1,26.6,NaN,NaN,...,NaN,NaN,18.0,NaN,NaN,NaN,900.0,NaN,NaN,NaN
27,109290.0,2.0,68.0,2.0,4.0,73.0,161.2,28.1,7.0,45.0,...,NaN,NaN,NaN,NaN,NaN,90.0,180.0,30.0,NaN,NaN
35,109298.0,2.0,68.0,1.0,3.0,90.2,162.5,34.2,NaN,NaN,...,NaN,NaN,NaN,120.0,NaN,NaN,120.0,NaN,1.0,NaN
37,109300.0,2.0,54.0,2.0,6.0,62.0,144.7,29.6,3.0,50.0,...,NaN,NaN,NaN,NaN,NaN,NaN,60.0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15549,124812.0,2.0,62.0,2.0,2.0,73.0,159.6,28.7,7.0,30.0,...,3.0,NaN,14.0,NaN,NaN,NaN,240.0,NaN,1.0,NaN
15551,124814.0,2.0,64.0,1.0,4.0,114.3,174.5,37.5,NaN,NaN,...,2.0,NaN,15.0,NaN,NaN,NaN,300.0,20.0,NaN,NaN
15552,124815.0,2.0,52.0,1.0,4.0,94.3,178.8,29.5,NaN,NaN,...,1.0,NaN,27.0,360.0,120.0,120.0,60.0,60.0,NaN,NaN
15554,124817.0,1.0,67.0,2.0,1.0,82.8,147.8,37.9,3.0,47.0,...,2.0,NaN,NaN,10.0,NaN,NaN,180.0,NaN,NaN,NaN


In [33]:
file_path = f"2017-2020/{files[1]}.xpt"
test = pd.read_sas(file_path, format="xport")
filtered = test.filter(items = codes)
filtered.rename(columns=labels_mapping, inplace=True)

patient_rows = filtered[filtered['patient'] == 109290.0]
patient_rows

# filtered.head()

,patient,age,gender,ethnicity
27,109290.0,68.0,2.0,4.0
